# Tutorial on ProfileOrdinal

In [1]:
from fractions import Fraction
import poisson_approval as pa

## A Profile and its Basic Properties

Create a profile:

In [2]:
profile = pa.ProfileOrdinal({'abc': Fraction(1, 10), 'bac': Fraction(6, 10), 'cab': Fraction(3, 10)})
profile

<abc: 1/10, bac: 3/5, cab: 3/10> (Condorcet winner: b)

Share of voters $abc$ (i.e. who prefer candidate $a$, then $b$, then $c$):

In [3]:
profile.abc

Fraction(1, 10)

Which rankings are in the profile?

In [4]:
profile.support_in_rankings

{abc, bac, cab}

Are all possible rankings in the profile?

In [5]:
profile.is_generic_in_rankings

False

Is one ranking shared by a majority of voters?

In [6]:
profile.has_majority_ranking

True

Is one candidate prefered by a majority of voters?

In [7]:
profile.has_majority_favorite

True

Is the profile single-peaked?

In [8]:
profile.is_single_peaked

True

Weighted majority graph:

In [9]:
profile.weighted_maj_graph

array([[0, Fraction(-1, 5), Fraction(2, 5)],
       [Fraction(1, 5), 0, Fraction(2, 5)],
       [Fraction(-2, 5), Fraction(-2, 5), 0]], dtype=object)

Condorcet winner(s):

In [10]:
profile.condorcet_winners

{'b'}

Does the profile have a Condorcet winner?

(1.0: Yes, 0.5: one or several weak Condorcet winners, 0.0: No)

In [11]:
profile.is_profile_condorcet

1.0

N.B.: All the above basic properties are defined not only for a ProfileOrdinal, but also for any Profile, such as ProfileTwelve or ProfileHistogram.

## Study a Strategy

Define a strategy:

In [12]:
strategy = pa.StrategyOrdinal({'abc': 'a', 'bac': 'ab', 'cab': 'c'})
strategy

<abc: a, bac: ab, cab: c>

Ballot of the voters with ranking $abc$:

In [13]:
strategy.abc

'a'

Tau vector (ballot shares) associated to the strategy in the given profile:

In [14]:
profile.tau(strategy)

<a: 1/10, ab: 3/5, c: 3/10> ==> a

Is the strategy an equilibrium for the given profile?

In [15]:
profile.is_equilibrium(strategy)

EquilibriumStatus.EQUILIBRIUM

Alternatively, as soon as you define a strategy, you can attach a profile to it. In that case, the strategy is considered from the point of view of its usage in the given profile. Thus you can write:

In [16]:
strategy = pa.StrategyOrdinal({'abc': 'a', 'bac': 'ab', 'cab': 'c'}, profile=profile)
strategy

<abc: a, bac: ab, cab: c> ==> a

In [17]:
strategy.tau

<a: 1/10, ab: 3/5, c: 3/10> ==> a

In [18]:
strategy.is_equilibrium

EquilibriumStatus.EQUILIBRIUM

## Study all Strategies

Analyze all possible strategies:

In [19]:
profile.analyzed_strategies

<bound method Profile.analyzed_strategies of ProfileOrdinal({'abc': Fraction(1, 10), 'bac': Fraction(3, 5), 'cab': Fraction(3, 10)})>

To access one of these strategies in particular:

In [20]:
profile.analyzed_strategies_ordinal.equilibria[0]

<abc: a, bac: b, cab: ac> ==> b

In [21]:
profile.analyzed_strategies_ordinal.utility_dependent[0]

<abc: ab, bac: b, cab: c> ==> b

In [22]:
profile.analyzed_strategies_ordinal.non_equilibria[0]

<abc: a, bac: b, cab: c> ==> b

The following functions compute probabilities. They are based on the assumption that the utility of voters $abc$ for their middle candidate $b$ is drawn uniformly between 0 and 1; and similarly for voters $acb$, $bac$, etc.

Probability that there exists an equilibrium:

(In the example below, the probability is 1 because in this profile, there exists some equilibria that do not depend on the utilities.)

In [23]:
profile.proba_equilibrium()

1

Probability that there exists an equilibrium where voters $abc$ cast a ballot $ab$:

In [24]:
def test_abc_vote_ab(strategy):
    return strategy.abc == 'ab'
profile.proba_equilibrium(test=test_abc_vote_ab)

0.13709468816628012

Probability that there exists an equilibrium where candidate $c$ wins:

In [25]:
def test_c_wins(strategy):
    return 'c' in strategy.winners
profile.proba_equilibrium(test=test_c_wins)

0

Distribution of the number of equilibria:

(In the example below, it is impossible that there is 0 or 1 equilibrium and positive probabilities that there are 2 or 3 equilibria.)

In [26]:
profile.distribution_equilibria()

array([0.        , 0.        , 0.86290531, 0.13709469])

You can access the distribution of equilibria, conditionally on a given test:

(In the example below, there are positive probabilities that there is 0 or 1 equilibrium.)

In [27]:
profile.distribution_equilibria(test=test_abc_vote_ab)

array([0.86290531, 0.13709469])

Distribution of the number of winners at equilibrium:

(In the example below, there are always 2 candidates, namely $a$ and $b$, that can win at equilibrium.)

In [28]:
profile.distribution_winners()

array([0., 0., 1., 0.])

You can access the distribution of winners, conditionally on a given test:

(In the example below, there can be 0 winner, i.e. when there is no equilibrium, or 1 winner.)

In [29]:
profile.distribution_winners(test=test_abc_vote_ab)

array([0.86290531, 0.13709469, 0.        , 0.        ])

## Weak orders

The profile can contains weak orders. Voters of the form ``'a>b~c'`` ("lovers") always vote for their top candidate, and voters of the form ``'a~b>c'`` ("haters") always vote for their two top candidates.

In [30]:
profile = pa.ProfileOrdinal({'abc': Fraction(3, 10), 'bac': Fraction(3, 10), 'c>a~b': Fraction(4, 10)})

In [31]:
profile.analyzed_strategies_ordinal

Equilibria:
None

Utility-dependent equilibria:
<abc: a, bac: ab> ==> a (FF)
<abc: ab, bac: b> ==> b (FF)

Non-equilibria:
<abc: a, bac: b> ==> c (BF)
<abc: ab, bac: ab> ==> a, b (FF)